# Chat models
source: https://python.langchain.com/docs/how_to/#chat-models

Langchain **chat models** are language models that use a sequence of messages as inputs and return messages as outputs (as opposed to using plain text). These are generally newer models.

## Setup

Install LLM models as langchain dependencies: 

```bash
uv pip install "langchain[anthropic]"
```

Ex: anthropic, openai, google-genai (gemini), google-vertexai, aws,  groq, cohere, langchain-nvidia-ai-endpoints (nvidia), fireworks, mistralai, together, langchain-ibm (WatsonX), databricks-langchain, langchain-xai, langchain-perplexity, langchain-deepseek, langchain-ollama, ...


See link above for model features.

## Fundamentals of chat models

In [ ]:
from dotenv import load_dotenv
import os, getpass

load_dotenv()
message = "hi! Tell me a joke."

# if not os.environ.get("GOOGLE_API_KEY"):
#   os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")


In [ ]:
# ---- OpenAI chat model example ----
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(openai_api_key=os.environ["GOOGLE_API_KEY"])
try:
    result = chat_model.predict(message)
except Exception as e:
    result = str(e)
    
print(result)

In [ ]:
# ---- Generic model example ----
from langchain.chat_models import init_chat_model

# Instantiation
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

# Invocation
llm.invoke("Hello, world!")

In [ ]:
# ---- Anthropic Chat Model Example ----
# Anthropic models: https://docs.anthropic.com/en/docs/models-overview
from langchain.chat_models import ChatAnthropic

# Instantiation
model = ChatAnthropic(model="claude-3-opus-20240229")
result = model.invoke(message)
print(f"Answer from Anthropic: {result.content}")

In [ ]:
# ---- Google Chat Model Example ----
# https://ai.google.dev/gemini-api/docs/models/gemini
from langchain_googlegenerativeai import ChatGoogleGenerativeAI

# Instantiation
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
result = model.invoke(message)
print(f"Answer from Google: {result.content}")

In [18]:
# ---- Ollama chat model example ----
from langchain_ollama import ChatOllama

# Instantiation
message = "hi! Tell me a joke about doctors."
model = ChatOllama(
    base_url="http://localhost:11434",
    model="llama3.2:3b",
    temperature=0,
    # other params...
)

# Invocation
result = model.invoke(message)
print(result.content)

Why did the doctor put a band-aid on the computer?

Because it had a virus! (get it?)


In [ ]:
from langchain.llms import Ollama

# Instantiation
# Note: ensure the tag is specified with the model  
# Ex: qwen2.5-coder:7b,  gpt-oss:20b, llama3.2:3b
model = Ollama(base_url="http://localhost:11434", model="llama3.2:3b", temperature=0.5)

# Invocation
message = "hi! Tell me a joke."
result = model.invoke(message)

# OBS! langchain.llms.Ollama is a chat invokation (returns string)
print(result)

Here's one:

What do you call a fake noodle?

An impasta!

Hope that made you smile! Do you want to hear another one?


## Chat model used for conversation (sequential inputs)

This introduces the message class and 3 types of messages:  
- SystemMessage: directed to the LLM
- HumanMessage: discussion from human
- AIMessage: discussion from AI LLM

In [23]:
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# model = ChatOpenAI(model="gpt-4o")
model = ChatOllama(base_url="http://localhost:11434", model="llama3.2:3b", temperature=0.)

# SystemMessage: Message for priming AI behavior, usually passed in as the first of a sequenc of input messages.
# HumanMessagse: Message from a human to the AI model.
messages = [
    SystemMessage(content="You are an impatient teacher. You are in a rush to finish your class. You do not explain topics in great detail and give very short answers ."),
    HumanMessage(content="What is linear algebra?"),
]

# Invoke the model with messages
result = model.invoke(messages)
print(f"--- Answer from AI: \n{result.content}")

print(f"\n--- Additional messages... ---\n")

# Additional interaction via contextual messages
messages.append(SystemMessage(content="You are now a patient teacher and want to help the student to understand maths so you explain topics in a pedagogic way."))
messages.append(AIMessage(content="Is this clear?"))
messages.append(HumanMessage(content="not really.."))

# Invoke the model with messages
result = model.invoke(messages)
print(f"--- Answer from AI: \n{result.content}")


--- Answer from AI: 
(sigh) Fine. Linear algebra. It's a branch of math that deals with vectors, matrices, and linear transformations. Now, move on. We don't have all day. Next question!

--- Additional messages... ---

--- Answer from AI: 
Don't worry, let me start again.

Linear Algebra is a branch of mathematics that deals with vectors and matrices. It's all about understanding how to manipulate these mathematical objects to solve problems in various fields like physics, engineering, computer science, and more.

Think of it like this: Vectors are like arrows on a graph, and Matrices are like tables of numbers that help us describe the relationships between these arrows. By studying Linear Algebra, we can learn how to perform operations with these vectors and matrices, like addition, multiplication, and finding their inverses.

It's a fundamental subject that helps us solve systems of equations, find eigenvalues and eigenvectors, and understand many other important concepts in mathem

## Chat history

In [ ]:
from dotenv import load_dotenv
from pprint import pprint
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

# Load environment variables from .env (in particular loads OPENAI_API_KEY)
load_dotenv()

# Instantiate a model
# model = ChatOpenAI(model="gpt-4o-mini")
model = ChatOllama(base_url="http://localhost:11434", model="llama3.2:3b", temperature=0.)

# Use a list to store messages
chat_history = []  

# Set an initial system message (optional)
system_message = SystemMessage(content="You are a helpful AI assistant.")
chat_history.append(system_message)

# Chat loop
while True:
    print('--- NEW ENTRY ---')
    query = input("You: ")
    print(f"{query}")
    if query.lower() == "exit":
        break
    chat_history.append(HumanMessage(content=query))  # Add user message

    # Get AI response using history
    result = model.invoke(chat_history)
    response = result.content
    chat_history.append(AIMessage(content=response))  # Add AI message

    print(f"AI: {response}")


print("\n\n---- Message History ----")
pprint(chat_history)


--- NEW ENTRY ---
AI: I'd be happy to help with your math question. What topic or concept would you like me to explain? Is it algebra, geometry, calculus, or something else?
--- NEW ENTRY ---
AI: Linear Algebra is a branch of mathematics that deals with:

* Vectors and vector operations (addition, scalar multiplication)
* Matrices and matrix operations (multiplication, inversion)
* Linear transformations and their properties
* Systems of equations and eigenvalues

Think of it as the study of how to manipulate and analyze linear relationships between variables.
--- NEW ENTRY ---
AI: If you have any more questions or need further clarification on linear algebra, feel free to ask!
--- NEW ENTRY ---


---- Message History ----
[SystemMessage(content='You are a helpful AI assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi, can you help with a math explanation.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'd be happy to help with your m

## Storing chat history in a database
source: https://python.langchain.com/docs/integrations/memory/

In [ ]:
# source: https://python.langchain.com/docs/integrations/memory/google_firestore_datastore/
from dotenv import load_dotenv
import redis
from langchain_google_memorystore_redis import MemorystoreChatMessageHistory
from langchain_openai import ChatOpenAI

load_dotenv()

# Connect to a Memorystore for Redis instance
# docker run -d --name redis -p 6379:6379 redis:alpine 
redis_client = redis.Redis(host='localhost', port=6379)
# redis_client = redis.from_url("redis://127.0.0.1:6379")
redis_key = "Session1"

# Initialize message history
message_history = MemorystoreChatMessageHistory(redis_client, session_id=redis_key)
print("Current Chat History:", message_history.messages)

# Initialize Chat Model
model = ChatOpenAI()
print("Start chatting with the AI. Type 'exit' to quit.")

while True:
    human_input = input("User: ")
    if human_input.lower() == "exit":
        break

    message_history.add_user_message(human_input)

    ai_response = model.invoke(message_history.messages)
    message_history.add_ai_message(ai_response.content)

    print(f"AI: {ai_response.content}")

message_history = MemorystoreChatMessageHistory(redis_client, session_id="session1")
print("Current Chat History:", message_history.messages)

# output:
# Current Chat History: [HumanMessage(content='this is a test', additional_kwargs={}, response_metadata={}), 
#                       HumanMessage(content='Tell me a joke', additional_kwargs={}, response_metadata={}),
#                       AIMessage(content="Why don't scientists trust atoms? Because they make up everything!", additional_kwargs={}, response_metadata={})
# ]